# InfluencerRank v8: Best Single Model (GAT + GRU)

**Model**: Heterogeneous GNN with GAT convolution + GRU

**Key Features**:
1. HeteroGNN with GAT layers
2. Layer concatenation: [F^(1), F^(2)] = 256 dims

In [1]:
import os
os.system('pip uninstall torch torchvision torchaudio torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv pyg-lib -y')

!pip install torch==2.3.1+cu121 torchvision==0.18.1+cu121 torchaudio==2.3.1+cu121 --index-url https://download.pytorch.org/whl/cu121

!pip install torch-scatter==2.1.2+pt23cu121 torch-sparse==0.6.18+pt23cu121 torch-cluster==1.6.3+pt23cu121 torch-spline-conv==1.2.2+pt23cu121 -f https://data.pyg.org/whl/torch-2.3.0+cu121.html

!pip install torch-geometric
!pip install pyg-lib -f https://data.pyg.org/whl/torch-2.3.0+cu121.html

import os
# os._exit(00)

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124


Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.0/781.0 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 102.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.

## 1. Setup and Imports

In [2]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import ndcg_score
import warnings
import time
import pickle
warnings.filterwarnings('ignore')

from torch_geometric.nn import SAGEConv, GATConv, HeteroConv
import torch_geometric.transforms as T

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

Using device: cuda
GPU: Tesla T4
GPU Memory: 15.83 GB


## 2. Configuration

In [3]:
# V3 GRAPHS (with actual edges!)
# GRAPH_DIR = 'graphs_enhanced_v3'  # Local
GRAPH_DIR = '/kaggle/input/graphsv3/graphs_enhanced_v3'  # Kaggle

# STRICT MASKING (17 features)
STRICT_MASK_INDICES = [
    11, 12, 13, 14, 15, 16, 17, 18, 19, 20,  # Temporal Trends (10)
    21, 22,                                  # Activity Rate & Consistency
    23, 24,                                  # Growth Features
    26, 27,                                  # Raw Monthly Counts
    36                                       # Sentiment Rate
]

print(f"✅ V8 Best Model (GAT+GRU)")
print(f"✅ Masking {len(STRICT_MASK_INDICES)} features")

# Architecture
INPUT_DIM = 37
GNN_HIDDEN = 128
GNN_OUT = 128
RNN_HIDDEN = 128
DROPOUT = 0.5

# Training
BATCH_SIZE = 64
LIST_SIZE = 10
LEARNING_RATE = 0.0015
NUM_EPOCHS = 50
WEIGHT_DECAY = 1e-5

# Temporal
TRAINING_MONTHS = 9
TARGET_MONTH = 9
SEED = 44

# Split
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

# Save directory
MODEL_SAVE_DIR = 'saved_models_v8_final'

✅ V8 Best Model (GAT+GRU)
✅ Masking 17 features


## 3. Load V3 Graphs

In [4]:
month_names = ["jan", "feb", "mar", "apr", "may", "jun", 
               "jul", "aug", "sep", "oct", "nov", "dec"]

print("Loading V3 graphs...")
all_graphs_data = []

for month_idx, month in enumerate(month_names):
    path = os.path.join(GRAPH_DIR, f"{month}_graph.pt")
    data = torch.load(path, weights_only=False)
    all_graphs_data.append(data)
    
    graph = data['graph']
    total_edges = sum(graph[edge_type].edge_index.shape[1] for edge_type in graph.edge_types)
    
    print(f"  {month.upper()}: {graph['influencer'].num_nodes} inf, "
          f"{graph['hashtag'].num_nodes} hashtag, {graph['user'].num_nodes} user, "
          f"{graph['object'].num_nodes} obj, {total_edges} edges")

print(f"\n✅ Loaded {len(all_graphs_data)} v3 heterogeneous graphs")

Loading V3 graphs...
  JAN: 2556 inf, 34139 hashtag, 17924 user, 80 obj, 210050 edges
  FEB: 2745 inf, 35493 hashtag, 19035 user, 80 obj, 224801 edges
  MAR: 2998 inf, 42116 hashtag, 24285 user, 80 obj, 289724 edges
  APR: 3258 inf, 46382 hashtag, 26540 user, 80 obj, 318269 edges
  MAY: 3549 inf, 52662 hashtag, 30100 user, 80 obj, 374002 edges
  JUN: 3831 inf, 56844 hashtag, 31715 user, 80 obj, 399204 edges
  JUL: 4136 inf, 62930 hashtag, 35677 user, 80 obj, 471521 edges
  AUG: 4465 inf, 70364 hashtag, 39673 user, 80 obj, 548174 edges
  SEP: 4808 inf, 74746 hashtag, 42989 user, 80 obj, 582115 edges
  OCT: 5180 inf, 84229 hashtag, 47669 user, 80 obj, 683653 edges
  NOV: 5545 inf, 89226 hashtag, 51248 user, 80 obj, 733528 edges
  DEC: 5973 inf, 97018 hashtag, 55221 user, 80 obj, 840515 edges

✅ Loaded 12 v3 heterogeneous graphs


## 4. Data Split

In [5]:
target_data = all_graphs_data[TARGET_MONTH]
all_influencers = list(target_data['maps']['influencer'].keys())
np.random.seed(42)
np.random.shuffle(all_influencers)

n = len(all_influencers)
n_train = int(TRAIN_RATIO * n)
n_val = int(VAL_RATIO * n)

train_influencers = set(all_influencers[:n_train])
val_influencers = set(all_influencers[n_train:n_train + n_val])
test_influencers = set(all_influencers[n_train + n_val:])

print(f"Split: Train={len(train_influencers)}, Val={len(val_influencers)}, Test={len(test_influencers)}")

train_influencers_list = list(train_influencers)
val_influencers_list = list(val_influencers)
test_influencers_list = list(test_influencers)

Split: Train=4144, Val=518, Test=518


## 5. Train-Only Normalization and Masking

In [6]:
print("Train-only normalization and STRICT leakage masking...")

# 1. Collect training features for scaler fitting
train_features = []
for month_idx in range(TRAINING_MONTHS):
    features = all_graphs_data[month_idx]['graph']['influencer'].x
    influencer_map = all_graphs_data[month_idx]['maps']['influencer']
    for inf_name in train_influencers:
        if inf_name in influencer_map:
            train_features.append(features[influencer_map[inf_name]].numpy())

# 2. Fit Scaler
scaler = StandardScaler()
scaler.fit(np.vstack(train_features))

print(f"Masking {len(STRICT_MASK_INDICES)} features to force RNN learning.")
print(f"Indices: {STRICT_MASK_INDICES}")

# 3. Apply Normalization AND Masking
for data_package in all_graphs_data:
    features = data_package['graph']['influencer'].x
    normalized_features = torch.FloatTensor(scaler.transform(features.numpy()))
    
    # APPLY MASK
    normalized_features[:, STRICT_MASK_INDICES] = 0.0
    
    data_package['graph']['influencer'].x = normalized_features

print(f"✅ Normalized and masked. Model must learn from structure and content.")

Train-only normalization and STRICT leakage masking...
Masking 17 features to force RNN learning.
Indices: [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 36]
✅ Normalized and masked. Model must learn from structure and content.


## 6. Prepare Heterogeneous Graphs

In [7]:
print("\nPreparing heterogeneous graphs for GPU...")
hetero_graphs = []

for month_idx in range(TRAINING_MONTHS):
    graph = all_graphs_data[month_idx]['graph']
    graph = T.ToUndirected()(graph)
    graph = graph.to(device)
    
    influencer_map = all_graphs_data[month_idx]['maps']['influencer']
    
    hetero_graphs.append({
        'graph': graph,
        'influencer_map': influencer_map
    })
    
    total_edges = sum(graph[edge_type].edge_index.shape[1] for edge_type in graph.edge_types)
    print(f"  Month {month_idx}: {total_edges} edges (undirected)")

print(f"\n✅ All {len(hetero_graphs)} heterogeneous graphs prepared on GPU")


Preparing heterogeneous graphs for GPU...
  Month 0: 420100 edges (undirected)
  Month 1: 449602 edges (undirected)
  Month 2: 579448 edges (undirected)
  Month 3: 636538 edges (undirected)
  Month 4: 748004 edges (undirected)
  Month 5: 798408 edges (undirected)
  Month 6: 943042 edges (undirected)
  Month 7: 1096348 edges (undirected)
  Month 8: 1164230 edges (undirected)

✅ All 9 heterogeneous graphs prepared on GPU


## 7. Model Architecture

In [8]:
class HeteroGNN(nn.Module):
    """Heterogeneous GNN with GAT convolution."""
    def __init__(self, in_channels, hidden_channels, out_channels, metadata, dropout=0.5):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        
        self.conv1 = HeteroConv({
            edge_type: GATConv((-1, -1), hidden_channels, add_self_loops=False, heads=1)
            for edge_type in metadata[1]
        }, aggr='sum')
        
        self.conv2 = HeteroConv({
            edge_type: GATConv((-1, -1), out_channels, add_self_loops=False, heads=1)
            for edge_type in metadata[1]
        }, aggr='sum')
        
        self.linear_proj = nn.Linear(in_channels, hidden_channels + out_channels)
        
    def forward(self, x_dict, edge_index_dict):
        edge_index_dict_filtered = {k: v for k, v in edge_index_dict.items() if v.shape[1] > 0}
        if len(edge_index_dict_filtered) == 0:
            return {key: self.linear_proj(x) for key, x in x_dict.items()}

        # Layer 1
        x_dict_1 = self.conv1(x_dict, edge_index_dict_filtered)
        x_dict_1 = {key: F.relu(x) for key, x in x_dict_1.items() if x is not None}
        x_dict_1 = {key: self.dropout(x) for key, x in x_dict_1.items()}
        
        if len(x_dict_1) == 0:
            return {key: self.linear_proj(x) for key, x in x_dict.items()}

        # Layer 2
        edge_index_dict_filtered_2 = {k: v for k, v in edge_index_dict_filtered.items() if k[0] in x_dict_1 and k[2] in x_dict_1}
        if len(edge_index_dict_filtered_2) == 0:
            x_dict_2 = {k: torch.zeros_like(v) for k, v in x_dict_1.items()}
        else:
            x_dict_2 = self.conv2(x_dict_1, edge_index_dict_filtered_2)
            x_dict_2 = {key: x for key, x in x_dict_2.items() if x is not None}

        return {k: torch.cat([x_dict_1[k], x_dict_2[k]], dim=1) for k in x_dict_1.keys() if k in x_dict_2}


class SimpleAttention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.attention_fc = nn.Linear(hidden_size, 1)
    
    def forward(self, hidden_states, lengths):
        batch_size, seq_len, _ = hidden_states.shape
        scores = self.attention_fc(hidden_states).squeeze(-1)
        mask = torch.arange(seq_len, device=hidden_states.device).expand(batch_size, -1)
        mask = mask < lengths.unsqueeze(1)
        scores = scores.masked_fill(~mask, -1e9)
        weights = F.softmax(scores, dim=1)
        return torch.bmm(weights.unsqueeze(1), hidden_states).squeeze(1)


class BestInfluencerModel(nn.Module):
    """Best performing model: GAT + GRU."""
    def __init__(self, input_dim, gnn_hidden, gnn_out, rnn_hidden, metadata, dropout=0.5):
        super().__init__()
        self.hetero_gnn = HeteroGNN(input_dim, gnn_hidden, gnn_out, metadata, dropout)
        
        gnn_concat_dim = gnn_hidden + gnn_out
        self.rnn = nn.GRU(gnn_concat_dim, rnn_hidden, num_layers=1, batch_first=True)
        self.attention = SimpleAttention(rnn_hidden)
        self.fc1 = nn.Linear(rnn_hidden, rnn_hidden // 2)
        self.fc2 = nn.Linear(rnn_hidden // 2, 1)
        self.dropout = nn.Dropout(dropout)
        
    def encode_graphs(self, hetero_graphs):
        embeddings = []
        for hg in hetero_graphs:
            graph = hg['graph']
            x_dict = {node_type: graph[node_type].x for node_type in graph.node_types if hasattr(graph[node_type], 'x')}
            edge_index_dict = graph.edge_index_dict if len(graph.edge_index_dict) > 0 else {}
            emb_dict = self.hetero_gnn(x_dict, edge_index_dict)
            embeddings.append(emb_dict['influencer'])
        return embeddings
    
    def forward_temporal(self, sequences, lengths):
        packed = pack_padded_sequence(sequences, lengths.cpu(), batch_first=True, enforce_sorted=False)
        rnn_output, _ = pad_packed_sequence(self.rnn(packed)[0], batch_first=True)
        context = self.attention(rnn_output, lengths.to(rnn_output.device))
        x = F.relu(self.fc1(context))
        return self.fc2(self.dropout(x)).squeeze(-1)

print("✅ Model architecture defined (GAT + GRU)")

✅ Model architecture defined (GAT + GRU)


## 8. Loss & Metrics

In [9]:
def listwise_ranking_loss(y_pred, y_true):
    """Listwise ranking loss."""
    pred_diff = y_pred.unsqueeze(1) - y_pred.unsqueeze(0)
    true_diff = y_true.unsqueeze(1) - y_true.unsqueeze(0)
    mask = (true_diff > 0).float()
    return (-F.logsigmoid(pred_diff) * mask).sum() / mask.sum().clamp(min=1)

def compute_ndcg_at_k(y_true, y_pred, k=50):
    if len(y_true) < 2: return 0.0
    return ndcg_score(np.array(y_true).reshape(1, -1), np.array(y_pred).reshape(1, -1), k=min(k, len(y_true)))

def get_ground_truth(influencer_names, target_data):
    ground_truth = []
    influencer_map = target_data['maps']['influencer']
    engagement_rates = target_data['ground_truth']['engagement_rate']
    for name in influencer_names:
        ground_truth.append(engagement_rates[influencer_map[name]].item() if name in influencer_map else 0.0)
    return torch.FloatTensor(ground_truth)

def sample_influencers(influencer_list, size):
    return [influencer_list[i] for i in np.random.choice(len(influencer_list), size, replace=False)] if len(influencer_list) >= size else influencer_list

print("✅ Loss & metrics defined")

✅ Loss & metrics defined


## 9. Training Loop

In [10]:
def train_single_model(hetero_graphs, train_list, val_list, test_list, all_graphs_data, seed=SEED):
    print(f"\n{'='*60}\nTraining Best Model (GAT + GRU) with seed {seed}\n{'='*60}")
    
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed(seed)
    
    metadata = hetero_graphs[0]['graph'].metadata()
    target_data = all_graphs_data[TARGET_MONTH]
    
    model = BestInfluencerModel(
        INPUT_DIM, GNN_HIDDEN, GNN_OUT, RNN_HIDDEN, metadata, DROPOUT
    ).to(device)
    
    # Initialize model with dummy forward pass to avoid lazy parameter issues
    print("Initializing model...")
    model.train()
    _ = model.encode_graphs(hetero_graphs)
    print(f"Model params: {sum(p.numel() for p in model.parameters()):,}")
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    
    best_val_ndcg = 0.0
    best_model_state = None
    
    for epoch in range(NUM_EPOCHS):
        model.train()
        
        num_batches = max(1, len(train_list) // (BATCH_SIZE * LIST_SIZE))
        for _ in range(num_batches):
            optimizer.zero_grad()
            embeddings = model.encode_graphs(hetero_graphs)
            
            batch_names = sample_influencers(train_list, LIST_SIZE * BATCH_SIZE)
            sequences, valid_names = [], []
            for name in batch_names:
                seq = []
                for m in range(TRAINING_MONTHS):
                    if name in hetero_graphs[m]['influencer_map']:
                        seq.append(embeddings[m][hetero_graphs[m]['influencer_map'][name]])
                if len(seq) > 0:
                    sequences.append(torch.stack(seq))
                    valid_names.append(name)
            
            if len(sequences) < 2: continue
            lengths = torch.LongTensor([s.shape[0] for s in sequences])
            padded = pad_sequence(sequences, batch_first=True)
            y_true = get_ground_truth(valid_names, target_data).to(device)
            y_pred = model.forward_temporal(padded, lengths)
            
            loss = listwise_ranking_loss(y_pred, y_true)
            loss.backward()
            optimizer.step()
            del embeddings
        
        # Validation
        model.eval()
        with torch.no_grad():
            val_emb = model.encode_graphs(hetero_graphs)
            val_seqs, val_names = [], []
            for name in val_list:
                seq = [val_emb[m][hetero_graphs[m]['influencer_map'][name]] for m in range(TRAINING_MONTHS) if name in hetero_graphs[m]['influencer_map']]
                if len(seq)>0: val_seqs.append(torch.stack(seq)); val_names.append(name)
            
            if len(val_seqs) > 0:
                val_pad = pad_sequence(val_seqs, batch_first=True)
                val_len = torch.LongTensor([s.shape[0] for s in val_seqs])
                val_pred = model.forward_temporal(val_pad, val_len).cpu().numpy()
                val_true = get_ground_truth(val_names, target_data).numpy()
                val_ndcg = compute_ndcg_at_k(val_true, val_pred, k=50)
                
                if val_ndcg > best_val_ndcg:
                    best_val_ndcg = val_ndcg
                    best_model_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
        
        if (epoch+1) % 1 == 0:
            print(f"  Epoch {epoch+1}: Val NDCG={val_ndcg:.4f}")

    # Load Best & Test
    model.load_state_dict(best_model_state)
    model.eval()
    with torch.no_grad():
        test_emb = model.encode_graphs(hetero_graphs)
        test_seqs, test_names = [], []
        for name in test_list:
            seq = [test_emb[m][hetero_graphs[m]['influencer_map'][name]] for m in range(TRAINING_MONTHS) if name in hetero_graphs[m]['influencer_map']]
            if len(seq)>0: test_seqs.append(torch.stack(seq)); test_names.append(name)
        
        test_pad = pad_sequence(test_seqs, batch_first=True)
        test_len = torch.LongTensor([s.shape[0] for s in test_seqs])
        test_pred = model.forward_temporal(test_pad, test_len).cpu().numpy()
        test_true = get_ground_truth(test_names, target_data).numpy()
        test_ndcg = compute_ndcg_at_k(test_true, test_pred, k=50)
        
        print(f"\n  🏁 Best Val NDCG: {best_val_ndcg:.4f}")
        print(f"  🎯 Test NDCG@50: {test_ndcg:.4f}")

    return {
        'model': model,
        'best_val_ndcg': best_val_ndcg,
        'test_ndcg': test_ndcg,
        'test_pred': test_pred,
        'test_true': test_true,
        'test_names': test_names,
        'seed': seed
    }

print("✅ Training loop defined")

✅ Training loop defined


## 10. Train Model

In [11]:
print("\n" + "="*60)
print("V8 BEST MODEL TRAINING (GAT + GRU)")
print("="*60)

total_start = time.time()

result = train_single_model(
    hetero_graphs,
    train_influencers_list, val_influencers_list, test_influencers_list,
    all_graphs_data
)

total_time = time.time() - total_start

print(f"\n{'='*60}")
print(f"✅ TRAINING COMPLETE in {total_time/60:.1f} minutes")
print(f"{'='*60}")


V8 BEST MODEL TRAINING (GAT + GRU)

Training Best Model (GAT + GRU) with seed 44
Initializing model...
Model params: 424,450
  Epoch 1: Val NDCG=0.5621
  Epoch 2: Val NDCG=0.6151
  Epoch 3: Val NDCG=0.5933
  Epoch 4: Val NDCG=0.6529
  Epoch 5: Val NDCG=0.6611
  Epoch 6: Val NDCG=0.6615
  Epoch 7: Val NDCG=0.6915
  Epoch 8: Val NDCG=0.6898
  Epoch 9: Val NDCG=0.6773
  Epoch 10: Val NDCG=0.6916
  Epoch 11: Val NDCG=0.6736
  Epoch 12: Val NDCG=0.6717
  Epoch 13: Val NDCG=0.6558
  Epoch 14: Val NDCG=0.6709
  Epoch 15: Val NDCG=0.6783
  Epoch 16: Val NDCG=0.6545
  Epoch 17: Val NDCG=0.6740
  Epoch 18: Val NDCG=0.6819
  Epoch 19: Val NDCG=0.6805
  Epoch 20: Val NDCG=0.6805
  Epoch 21: Val NDCG=0.6636
  Epoch 22: Val NDCG=0.6571
  Epoch 23: Val NDCG=0.6656
  Epoch 24: Val NDCG=0.6639
  Epoch 25: Val NDCG=0.6478
  Epoch 26: Val NDCG=0.6747
  Epoch 27: Val NDCG=0.6881
  Epoch 28: Val NDCG=0.6798
  Epoch 29: Val NDCG=0.6570
  Epoch 30: Val NDCG=0.6854
  Epoch 31: Val NDCG=0.6731
  Epoch 32: Val

## 11. Save Model

In [12]:
# Create save directory
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

# Save model
torch.save({
    'model_state_dict': result['model'].state_dict(),
    'config': {
        'INPUT_DIM': INPUT_DIM,
        'GNN_HIDDEN': GNN_HIDDEN,
        'GNN_OUT': GNN_OUT,
        'RNN_HIDDEN': RNN_HIDDEN,
        'DROPOUT': DROPOUT,
        'STRICT_MASK_INDICES': STRICT_MASK_INDICES,
    },
    'results': {
        'test_ndcg': result['test_ndcg'],
        'val_ndcg': result['best_val_ndcg'],
        'seed': result['seed'],
    }
}, f'{MODEL_SAVE_DIR}/model.pt')

# Save scaler
with open(f'{MODEL_SAVE_DIR}/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Save data split
with open(f'{MODEL_SAVE_DIR}/data_split.pkl', 'wb') as f:
    pickle.dump({
        'train': train_influencers_list,
        'val': val_influencers_list,
        'test': test_influencers_list,
    }, f)

print(f"✅ Model saved to {MODEL_SAVE_DIR}/")

✅ Model saved to saved_models_v8_final/


## 12. Results

In [13]:
print(f"\n{'='*60}")
print("FINAL RESULTS")
print(f"{'='*60}")
print(f"  Model: GAT + GRU")
print(f"  Seed: {result['seed']}")
print(f"  Val NDCG@50: {result['best_val_ndcg']:.4f}")

print(f"  🎯 Test NDCG@50: {result['test_ndcg']:.4f}")
print(f"  Gap to paper: {0.72 - result['test_ndcg']:.4f}")

print(f"{'='*60}")


FINAL RESULTS
  Model: GAT + GRU
  Seed: 44
  Val NDCG@50: 0.7153
  🎯 Test NDCG@50: 0.7133
  Gap to paper: 0.0067


## 13. Top 50 Influencers

In [14]:
# Create ranking dataframe
ranking_df = pd.DataFrame({
    'influencer': result['test_names'],
    'predicted_score': result['test_pred'],
    'actual_engagement': result['test_true']
})

# Sort by predicted score
ranking_df = ranking_df.sort_values('predicted_score', ascending=False).reset_index(drop=True)
ranking_df['rank'] = range(1, len(ranking_df) + 1)

# Display top 50
top_50 = ranking_df.head(50)[['rank', 'influencer', 'predicted_score', 'actual_engagement']]

print("\n" + "="*80)
print("TOP 50 INFLUENCERS (Predicted by Model)")
print("="*80 + "\n")
print(top_50.to_string(index=False))
print("\n" + "="*80)


TOP 50 INFLUENCERS (Predicted by Model)

 rank            influencer  predicted_score  actual_engagement
    1         annakapiteijn         3.938305           0.132770
    2          bailee_denny         3.508782           0.098315
    3 alexandra.kirchberger         3.406746           0.109142
    4               chlo__j         3.321176           0.107476
    5        em_ilylonergan         3.249031           0.192504
    6           barbspatino         3.087434           0.097757
    7           amanda.paul         3.021748           0.094798
    8           dianaluchin         2.992497           0.218158
    9         brennanfoster         2.723327           0.074667
   10          adentrostyle         2.681482           0.086996
   11          ellietindall         2.670385           0.099074
   12             ellenbuui         2.661301           0.123982
   13      ceciliadelacruz7         2.646411           0.146299
   14         classiccarlos         2.632510           0.12573

## 14. Full Test Set with Rank Classes

In [15]:
# Add rank class
def get_rank_class(rank):
    if rank <= 10:
        return 'Top 10'
    elif rank <= 50:
        return 'Top 50'
    elif rank <= 100:
        return 'Top 100'
    elif rank <= 200:
        return 'Top 200'
    else:
        return 'Other'

ranking_df['rank_class'] = ranking_df['rank'].apply(get_rank_class)

# Full ranking
full_ranking = ranking_df[['rank', 'influencer', 'predicted_score', 'actual_engagement', 'rank_class']]

print(f"\n{'='*90}")
print(f"FULL TEST SET RANKINGS ({len(full_ranking)} influencers)")
print(f"{'='*90}\n")
print(full_ranking.to_string(index=False))
print(f"\n{'='*90}")

# Class distribution
print("\nRank Class Distribution:")
print(full_ranking['rank_class'].value_counts().sort_index())


FULL TEST SET RANKINGS (474 influencers)

 rank                    influencer  predicted_score  actual_engagement rank_class
    1                 annakapiteijn         3.938305           0.132770     Top 10
    2                  bailee_denny         3.508782           0.098315     Top 10
    3         alexandra.kirchberger         3.406746           0.109142     Top 10
    4                       chlo__j         3.321176           0.107476     Top 10
    5                em_ilylonergan         3.249031           0.192504     Top 10
    6                   barbspatino         3.087434           0.097757     Top 10
    7                   amanda.paul         3.021748           0.094798     Top 10
    8                   dianaluchin         2.992497           0.218158     Top 10
    9                 brennanfoster         2.723327           0.074667     Top 10
   10                  adentrostyle         2.681482           0.086996     Top 10
   11                  ellietindall         